Carga de los códigos SNOMED-CT.

In [ ]:
import os
f = open('der2_Refset_ProblemasSaludAHSpainExtensionSnapshot_ES_20221201.txt')
#for line in f:
#  print(line.strip())

In [ ]:
terminos_snomedct_problemassaludAH = []
cont = 0
for row in f:
  values = row.strip().split("\t")
  #id	effectiveTime	active	moduleId	refsetId	referencedComponentId	term
  #print(values)
  termino = {}
  termino["id"] = values[0];
  termino["effectiveTime"] = values[1];
  termino["active"] = values[2];
  termino["moduleId"] = values[3];
  termino["refsetId"] = values[4];
  termino["referencedComponentId"] = values[5];
  termino["term"] = values[6];
  termino["num_palabras"] = len(values[6].split(" "));
  #print(termino)
  terminos_snomedct_problemassaludAH.append(termino)

f.close();


Carga de modelo y obtención de embeddings de las palabras (es necesario preprocesar el texto)

In [ ]:
#!pip install gensim
from gensim.models import FastText
#El fichero .vec, nos vale sólo para palabras exactas
#Necesitamos usar el modelo .bin, para poder extraer el embedding de palabras nuevas
def loadmodel():
  model = FastText.load_fasttext_format('drive/MyDrive/clinic_es.bin')
  return model
model = loadmodel()

In [ ]:
for termino in terminos_snomedct_problemassaludAH:
  termino["embedding"] = model.wv.get_vector(termino["term"])

Prueba con un texto específico

In [ ]:
text = "El paciente presenta un abdomen distendido, y no es capaz de volverse loco"
tokens = text.split(" ");
print(tokens)


['El', 'paciente', 'presenta', 'un', 'abdomen', 'distendido,', 'y', 'no', 'es', 'capaz', 'de', 'volverse', 'loco']


Buscamos, para cada palabra la equivalencia terminológica y la mostramos

In [ ]:
for token in tokens:
  vector_A = model.wv.get_vector(token)
  for termino in terminos_snomedct_problemassaludAH:
    vector_B = termino["embedding"]
    cosine = np.dot(vector_A,vector_B)/(norm(vector_A)*norm(vector_B))
    if cosine > 0.80:
      print(token, " ", termino["term"], " Cosine Similarity:", cosine)


paciente   cuidado domiciliario del paciente  Cosine Similarity: 0.8024697
presenta   presentación pélvica  Cosine Similarity: 0.86765575
presenta   presentación pélvica anómala convertida exitosamente en presentación cefálica  Cosine Similarity: 0.83078706
distendido,   abdomen distendido  Cosine Similarity: 0.86112595


Guardamos en BBDD (esto no funciona en colab en la nube)

In [ ]:
#!pip install mysql-connector-python
import mysql.connector
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="mysql"
)
print(mydb)

DatabaseError: ignored

In [ ]:

mycursor = mydb.cursor()

mycursor.execute("CREATE DATABASE nlp_reports")
#id	effectiveTime	active	moduleId	refsetId	referencedComponentId	term
mycursor.execute("CREATE TABLE terminos_snomedct_problemassaludAH (id INT AUTO_INCREMENT PRIMARY KEY, effectiveTime VARCHAR(255), active VARCHAR(255), moduleId VARCHAR(255), refsetId VARCHAR(255), referencedComponentId VARCHAR(255), term VARCHAR(255), embedding VARCHAR(4000))")

In [ ]:
import json
#s = json.dumps(vector)
for termino in terminos_snomedct_problemassaludAH:
   mycursor.execute("INSERT INTO terminos_snomedct_problemassaludAH(id, effectiveTime, active, moduleId, refsetId, referencedComponentId, term, embedding) VALUES (%s,%s,...", (termino["id"], termino["effectiveTime"], termino["active"], termino["moduleId"], termino["refsetId"], termino["referencedComponentId"], termino["term"], termino["embedding"]))

A partir de aquí pruebas.

In [ ]:

# import required libraries
import numpy as np
from numpy.linalg import norm

A = model.wv.get_vector("distendido")
B = terminos_snomedct_problemassaludAH[1]["embedding"]

# compute cosine similarity
cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

Cosine Similarity: 0.8997975


In [ ]:
termino = terminos_snomedct_problemassaludAH[1]
print( termino["term"])

abdomen distendido


In [ ]:
termino["embedding"]

array([-3.41379493e-01,  8.51220787e-01,  1.97412252e-01, -1.13405132e+00,
        6.47886038e-01,  1.86931849e-01,  3.61175567e-01, -1.28212929e+00,
       -5.30260384e-01,  1.12864792e-01,  7.03634024e-02, -1.15307122e-01,
       -2.09871959e-02, -2.57505268e-01, -9.38366205e-02,  8.03253174e-01,
       -5.14225841e-01,  4.13851649e-01,  9.18954372e-01,  2.58679241e-01,
        2.99320877e-01, -3.65961552e-01,  6.31190598e-01, -8.59170675e-01,
       -5.97080648e-01,  2.79198170e-01,  2.81931423e-02, -1.17702079e+00,
       -5.98247409e-01, -7.17386425e-01,  3.12720798e-02,  1.11395383e+00,
        2.65944481e-01,  1.29806966e-01, -1.97782554e-02,  6.17558241e-01,
        4.67322022e-01,  1.75810909e+00,  3.73224556e-01, -6.67529702e-01,
        9.69988167e-01,  1.03337479e+00, -5.92100263e-01,  5.20665288e-01,
        3.72344494e-01,  7.10983753e-01, -4.01026636e-01,  4.44256663e-01,
       -4.41327363e-01, -6.60625339e-01, -2.97578603e-01, -3.63338649e-01,
        9.39562976e-01,  